In [14]:
import pandas as pd
import numpy as np

from warehouse_modeling.induced_backorder_cost import *
from warehouse_modeling.lead_time_approximation import *
from warehouse_modeling.warehouse_optimization import *
from warehouse_modeling.warehouse_demand_modeling import *

from single_echelon_utils.inventory_level_computation import *
from single_echelon_utils.service_level_computation import *
from single_echelon_utils.dealer_optimization import *

from utils import *

## INDATA
First, read indata from a specified excel file and sheet.

In [15]:
excel_path = "/Users/jakobbengtsson/Desktop/testdata_exjobb.xlsx"
indata_sheet = "indata"
indataDF = pd.read_excel(excel_path,indata_sheet)
outdataDF = indataDF.copy()
indataDF

,Installation id,Type,Name,Transport time,Q,Holding cost,Target item fill rate,Demand type,Demand mean,Demand stdev,Demand variance
0,1,RDC,Johannesburg,10,40,1,NaN,NaN,NaN,NaN,NaN
1,2,Dealer,Deal1,10,10,1,0.95,NBD,8.0,3.0,9.00
2,3,Dealer,Deal2,3,10,1,0.95,NBD,4.0,2.1,4.41
3,4,Dealer,Deal3,2,10,1,0.95,NBD,4.0,2.1,4.41
4,5,Dealer,Deal4,4,10,1,0.95,NBD,3.0,1.8,3.24
5,6,Dealer,Deal5,5,10,1,0.95,NBD,3.0,1.8,3.24


In [16]:
Q_dealer_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Q").to_numpy()
mu_dealer_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Demand mean").to_numpy()
sigma_dealer_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Demand stdev").to_numpy()
demand_type_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Demand type").to_numpy()
Q_subbatch_size = find_smallest_divisor(Q_dealer_arr)
L_wh = float(indataDF.get(indataDF["Type"]=="RDC").get("Transport time"))

In [17]:

rdc_f_u_probability_array, wh_dist, mu_L, sigma2_L = warehouse_subbatch_demand_probability_array(Q_dealer_arr, mu_dealer_arr, 
    sigma_dealer_arr, demand_type_arr, L_wh, Q_subbatch_size)

outdataDF.loc[outdataDF["Type"] == "RDC","Demand type"] = wh_dist
outdataDF.loc[outdataDF["Type"] == "RDC","Lead time demand mean"] = mu_L * Q_subbatch_size
outdataDF.loc[outdataDF["Type"] == "RDC","Lead time demand stdev"] = math.sqrt(sigma2_L) * Q_subbatch_size
outdataDF.loc[outdataDF["Type"] == "RDC","Demand mean"] = mu_L * Q_subbatch_size/L_wh
outdataDF.loc[outdataDF["Type"] == "RDC","Demand stdev"] = math.sqrt(sigma2_L) * Q_subbatch_size/L_wh

outdataDF

/Users/jakobbengtsson/Library/Mobile Documents/com~apple~CloudDocs/Documents/Master_thesis/single_echelon_utils/demand_models.py:194: RuntimeWarning: overflow encountered in double_scalars
  P_D_k *= (r + temp)


,Installation id,Type,Name,Transport time,Q,Holding cost,Target item fill rate,Demand type,Demand mean,Demand stdev,Demand variance,Lead time demand mean,Lead time demand stdev
0,1,RDC,Johannesburg,10,40,1,NaN,NBD,22.0,inf,NaN,220.0,inf
1,2,Dealer,Deal1,10,10,1,0.95,NBD,8.0,3.0,9.00,NaN,NaN
2,3,Dealer,Deal2,3,10,1,0.95,NBD,4.0,2.1,4.41,NaN,NaN
3,4,Dealer,Deal3,2,10,1,0.95,NBD,4.0,2.1,4.41,NaN,NaN
4,5,Dealer,Deal4,4,10,1,0.95,NBD,3.0,1.8,3.24,NaN,NaN
5,6,Dealer,Deal5,5,10,1,0.95,NBD,3.0,1.8,3.24,NaN,NaN


RDC reorder-point optimization

In [18]:
h_dealer_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Holding cost").to_numpy()
fill_rate_target_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Target item fill rate").to_numpy()
p_dealer_arr = fill_rate_target_arr*h_dealer_arr/(np.ones_like(fill_rate_target_arr)-fill_rate_target_arr)
l_dealer_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Transport time").to_numpy()
mu_wh = mu_L/L_wh * Q_subbatch_size

beta_list = []
for h,Q,p,l,my,sigma in zip(h_dealer_arr,Q_dealer_arr,p_dealer_arr,l_dealer_arr,mu_dealer_arr,sigma_dealer_arr):
    beta_list.append(induced_backorder_cost_opt(h,Q,p,l,my,sigma))

beta_arr = np.array(beta_list)

beta_rdc = weighting_backorder_cost(mu_dealer_arr,mu_wh,beta_arr)
print(f"Optimal weighted induced backorder cost at the warehouse is: {beta_rdc}, betas are: {beta_arr}")



Optimal weighted induced backorder cost at the warehouse is: 0.2494190290271901, betas are: [0.11777843 0.31238498 0.38565993 0.30905539 0.27521514]


In [19]:
h_rdc = float(indataDF.get(indataDF["Type"] == "RDC").get("Holding cost"))
Q_0 = int(indataDF.get(indataDF["Type"] == "RDC").get("Q"))

R_0 = warehouse_optimization(Q_subbatch_size,Q_0,rdc_f_u_probability_array,h_rdc,beta_rdc)
outdataDF.loc[outdataDF["Type"] == "RDC","R optimal"] = R_0
outdataDF


Starting optimizing, R = 0, c = 205.0, c+1 = 215.0
Doing downwards optimizing, R = -1, c = 195.0, c+1 = 205.0
Doing downwards optimizing, R = -2, c = 185.3123547572568, c+1 = 195.0
Doing downwards optimizing, R = -3, c = 175.9370642717704, c+1 = 185.3123547572568
Doing downwards optimizing, R = -4, c = 166.8741285435408, c+1 = 175.9370642717704
Doing downwards optimizing, R = -5, c = 158.123547572568, c+1 = 166.8741285435408
Doing downwards optimizing, R = -6, c = 149.68532135885195, c+1 = 158.123547572568
Doing downwards optimizing, R = -7, c = 141.55944990239274, c+1 = 149.68532135885195
Doing downwards optimizing, R = -8, c = 133.74593320319033, c+1 = 141.55944990239274
Doing downwards optimizing, R = -9, c = 126.24477126124471, c+1 = 133.74593320319033
Doing downwards optimizing, R = -10, c = 119.0559640765559, c+1 = 126.24477126124471
Doing downwards optimizing, R = -11, c = 112.17951164912387, c+1 = 119.0559640765559
Doing downwards optimizing, R = -12, c = 105.61541397894864, c+

,Installation id,Type,Name,Transport time,Q,Holding cost,Target item fill rate,Demand type,Demand mean,Demand stdev,Demand variance,Lead time demand mean,Lead time demand stdev,R optimal
0,1,RDC,Johannesburg,10,40,1,NaN,NBD,22.0,inf,NaN,220.0,inf,-33.0
1,2,Dealer,Deal1,10,10,1,0.95,NBD,8.0,3.0,9.00,NaN,NaN,NaN
2,3,Dealer,Deal2,3,10,1,0.95,NBD,4.0,2.1,4.41,NaN,NaN,NaN
3,4,Dealer,Deal3,2,10,1,0.95,NBD,4.0,2.1,4.41,NaN,NaN,NaN
4,5,Dealer,Deal4,4,10,1,0.95,NBD,3.0,1.8,3.24,NaN,NaN,NaN
5,6,Dealer,Deal5,5,10,1,0.95,NBD,3.0,1.8,3.24,NaN,NaN,NaN


In [20]:
W = waiting_time(negative_inventory(Q_subbatch_size,Q_0,R_0,rdc_f_u_probability_array),L_wh,mu_L,Q_subbatch_size)
outdataDF.loc[outdataDF["Type"]== "Dealer", "Waiting time"] = W
outdataDF.loc[outdataDF["Type"] == "Dealer", "Lead time"] = outdataDF.get(outdataDF["Type"]== "Dealer").get("Transport time").to_numpy() + W
outdataDF.loc[outdataDF["Type"] == "Dealer", "Lead time demand mean"] = outdataDF.get(outdataDF["Type"]== "Dealer").get("Lead time").to_numpy()*outdataDF.get(outdataDF["Type"]== "Dealer").get("Demand mean").to_numpy()
outdataDF.loc[outdataDF["Type"] == "Dealer", "Lead time demand stdev"] =outdataDF.get(outdataDF["Type"]== "Dealer").get("Lead time").to_numpy()*outdataDF.get(outdataDF["Type"]== "Dealer").get("Demand stdev").to_numpy()
outdataDF

,Installation id,Type,Name,Transport time,Q,Holding cost,Target item fill rate,Demand type,Demand mean,Demand stdev,Demand variance,Lead time demand mean,Lead time demand stdev,R optimal,Waiting time,Lead time
0,1,RDC,Johannesburg,10,40,1,NaN,NBD,22.0,inf,NaN,220.0,inf,-33.0,NaN,NaN
1,2,Dealer,Deal1,10,10,1,0.95,NBD,8.0,3.0,9.00,128.0,48.0,NaN,6.0,16.0
2,3,Dealer,Deal2,3,10,1,0.95,NBD,4.0,2.1,4.41,36.0,18.9,NaN,6.0,9.0
3,4,Dealer,Deal3,2,10,1,0.95,NBD,4.0,2.1,4.41,32.0,16.8,NaN,6.0,8.0
4,5,Dealer,Deal4,4,10,1,0.95,NBD,3.0,1.8,3.24,30.0,18.0,NaN,6.0,10.0
5,6,Dealer,Deal5,5,10,1,0.95,NBD,3.0,1.8,3.24,33.0,19.8,NaN,6.0,11.0


In [21]:
opt_dealer_list = []
L_dealer_arr = outdataDF.get(outdataDF["Type"] == "Dealer").get("Lead time")
for Q,L_est,fill_rate_target,demand_type,mu,sigma in zip(Q_dealer_arr,L_dealer_arr,fill_rate_target_arr,demand_type_arr, mu_dealer_arr,sigma_dealer_arr):
    print(demand_type, mu, sigma)
    opt_dealer_list.append(dealer_R_optimization(Q,L_est,fill_rate_target,demand_type,mu,demand_variance = math.pow(sigma,2)))

R_opt_dealer_list,fill_rate_dealer_list,exp_stock_on_hand_list = [],[],[]
for tup in opt_dealer_list:
    R_opt_dealer_list.append(tup[1])
    fill_rate_dealer_list.append(tup[2])
    exp_stock_on_hand_list.append(tup[3])

outdataDF.loc[outdataDF["Type"] == "Dealer", "R optimal"] = R_opt_dealer_list
outdataDF.loc[outdataDF["Type"] == "Dealer", "Realized item fill rate"] = fill_rate_dealer_list
outdataDF.loc[outdataDF["Type"] == "Dealer", "Expected stock on hand"] = exp_stock_on_hand_list

NBD 8.0 3.0
NBD 4.0 2.1
NBD 4.0 2.1
NBD 3.0 1.8
NBD 3.0 1.8


/Users/jakobbengtsson/Library/Mobile Documents/com~apple~CloudDocs/Documents/Master_thesis/single_echelon_utils/demand_models.py:194: RuntimeWarning: overflow encountered in double_scalars
  P_D_k *= (r + temp)
/Users/jakobbengtsson/Library/Mobile Documents/com~apple~CloudDocs/Documents/Master_thesis/single_echelon_utils/dealer_optimization.py:88: RuntimeWarning: invalid value encountered in double_scalars
  exp_stock_on_hand += i*p_IL


In [22]:
outdataDF

,Installation id,Type,Name,Transport time,Q,Holding cost,Target item fill rate,Demand type,Demand mean,Demand stdev,Demand variance,Lead time demand mean,Lead time demand stdev,R optimal,Waiting time,Lead time,Realized item fill rate,Expected stock on hand
0,1,RDC,Johannesburg,10,40,1,NaN,NBD,22.0,inf,NaN,220.0,inf,-33.0,NaN,NaN,NaN,NaN
1,2,Dealer,Deal1,10,10,1,0.95,NBD,8.0,3.0,9.00,128.0,48.0,10.0,6.0,16.0,inf,NaN
2,3,Dealer,Deal2,3,10,1,0.95,NBD,4.0,2.1,4.41,36.0,18.9,10.0,6.0,9.0,0.953895,12.618782
3,4,Dealer,Deal3,2,10,1,0.95,NBD,4.0,2.1,4.41,32.0,16.8,10.0,6.0,8.0,0.961007,12.592276
4,5,Dealer,Deal4,4,10,1,0.95,NBD,3.0,1.8,3.24,30.0,18.0,10.0,6.0,10.0,0.953330,11.609140
5,6,Dealer,Deal5,5,10,1,0.95,NBD,3.0,1.8,3.24,33.0,19.8,10.0,6.0,11.0,0.960723,12.593488
